<a href="https://colab.research.google.com/github/JoseMonteiroSantos/KNN-Recomendacoes/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementação do KNN

In [ ]:
from collections import Counter
import math

In [ ]:
def knn(data, x, k, distance_function, choice_function):
    neighbor_distances_and_indices = []
    
    # Calcula a distância entre x e as amostras da base de treinamento
    for index, example in enumerate(data):
        distance = distance_function(example[:-1], x) # a última coluna é a classe ou alvo, por isso não é usada no cálculo
        
        # Armazena a distância e o índice da amostra
        neighbor_distances_and_indices.append((distance, index))
    
    # Ordena as amostras por distância
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # Pega as k primeiras amostras (ou seja, as mais próximas)
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # Pega as classes dos k vizinhos
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]

    # Se for um problema de regressão, choice_function = mean
    # Se for um problema de classificação, choice_function = mode
    return k_nearest_distances_and_indices , choice_function(k_nearest_labels)

In [ ]:
def mean(labels):
    return sum(labels) / len(labels)

In [ ]:
def mode(labels):
    return Counter(labels).most_common(1)[0][0]

In [ ]:

def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

#Teste de Regressão
Dados os registros abaixo de altura (em polegadas) e peso (em libras), qual seria o peso aproximado de uma pessoa com 60 polegadas, com base nas 3 pessoas mais próximas?

In [ ]:
'''
    # Regression Data
    # 
    # Coluna 0: altura (polegadas)
    # Coluna 1: peso (libras)
    '''
reg_data = [
       [65.75, 112.99],
       [71.52, 136.49],
       [69.40, 153.03],
       [68.22, 142.34],
       [67.79, 144.30],
       [68.70, 123.30],
       [69.80, 141.49],
       [70.01, 136.46],
       [67.90, 112.37],
       [66.49, 127.45],
]
    

reg_query = [60]
reg_k_nearest_neighbors, reg_prediction = knn(
    reg_data, reg_query, k=3, distance_function=euclidean_distance, choice_function=mean
)

print('Altura aproximada:' , reg_prediction)

Altura aproximada: 128.24666666666667


#Teste de Classificação
Dado um conjunto de amostras defina se uma pessoa de 33 anos gosta ou não de abacaxi em sua pizza

In [ ]:
clf_data = [
  [22, 1],
  [23, 1],
  [21, 1],
  [18, 1],
  [19, 1],
  [25, 0],
  [27, 0],
  [29, 0],
  [31, 0],
  [45, 0],
]

clf_query = [33]
clf_k_nearest_neighbors, clf_prediction = knn(
    clf_data, clf_query, k=3, distance_function=euclidean_distance, choice_function=mode
)

if clf_prediction == 1 :
  print('Gosta de abacaxi')
else:
  print('Não gosta de abacaxi')

Não gosta de abacaxi


#KNN aplicado a Sistemas de Recomendação
Utilizando uma base extraída do IMDB, determinar 5 filme que se parecem com o filme mais recente do Homem Aranha

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving movies_recommendation_data.csv to movies_recommendation_data.csv


In [ ]:
def recommend_movies(movie_query, k_recommendations):
    raw_movies_data = []
    with open('movies_recommendation_data.csv', 'r') as md:
        # Descarta a primeira linha (cabeçalho)
        next(md)

        # Cria estrutura de dados
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_movies_data.append(data_row)

    # Prepare the data for use in the knn algorithm by picking
    # the relevant columns and converting the numeric columns
    # to numbers since they were read in as strings
    movies_recommendation_data = []
    for row in raw_movies_data:
        data_row = list(map(float, row[2:]))
        movies_recommendation_data.append(data_row)

    # Use the KNN algorithm to get the 5 movies that are most
    # similar to The Post.
    recommendation_indices, _ = knn(
        movies_recommendation_data, movie_query, k=k_recommendations,
        distance_function=euclidean_distance, choice_function=lambda x: None
    )

    movie_recommendations = []
    for _, index in recommendation_indices:
        movie_recommendations.append(raw_movies_data[index])

    return movie_recommendations

In [ ]:
from operator import itemgetter

spiderman = [8.3, 0, 0, 0, 0, 0, 0, 0, 0] # feature vector for The Post
recommended_movies = recommend_movies(movie_query=spiderman, k_recommendations=5)

sorted_movies = sorted(recommended_movies, key=itemgetter(2), reverse=True) # ordena pela nota do IMDB

for recommendation in sorted_movies:
  print(recommendation[1])

Inception
Spirited Away
The Avengers
The Terminator
Avatar
